# Speed Skydiving Analysis and Scoring 2019

Analyze one or more FlySight files with speed skydiving data.

This document implements scoring techniques compatible with the FAI World Air Sports Federation [Speed Skydiving Competition Rules, 2019 Edition](https://www.fai.org/sites/default/files/documents/2019_ipc_cr_speedskydiving.pdf) (PDF, 428 KB).

## Environment setup

In [6]:
from ssscore import COURSE_END
from ssscore import DEG_IN_RAD
from ssscore import VALID_MSL

import dateutil.parser
import math
import os
import os.path
import shutil

import pandas as pd

import ssscore

### Known drop zones AMSL in meters

The `ssscore` module defines these altitudes; the DZ name corresponds to the symbolic constant, e.g. Bay Area Skydiving ::= `BAY_AREA_SKYDIVING`.  The altitudes were culled from public airport information available on the Worldwide Web.

* Bay Area Skydiving = 23.90
* SkyDance Skydiving = 30.48
* Chicagoland Skydiving Center = 238.00
* Skydive Arizona = 460.60
* Skydive Algarve = 2.00
* Skydive Fano = 54.00
* Skydive Saulgau = 580.00
* Skydive Flanders Zwartberg = 85.00
* Skydive Buzz = 256.00

In [7]:
import ssscore.elevations as elevations # expressed in meters

DZ_AMSL = elevations.BAY_AREA_SKYDIVING

## FlySight data sources

1. Copy the FlySight `Tracks/YY-MM-dd` directory of interest to the `DATA_LAKE` directory;
   the `DATA_LAKE` can also be an external mount, a Box or Dropbox share, anything that 
   can be mapped to a directory -- even a whole drive!
1. Make a list of all CSV files in the FlySight `DATA_LAKE`
1. Move the CSV to the `DATA_SOURCE` bucket directory

### Define the data lake and data source

![Data sources diagram](images/SSScoring-data-sources.png)
<a id="l_data-def"></a>

In [8]:
DATA_LAKE   = './data-lake'
DATA_SOURCE = './data-sources/ciurana'

ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

moved 0 CSV files from data lake [./data-lake] to data source [./data-sources/ciurana]


## Top speed and pitch analysis on a single data file

User selects a valid FlySight data path and file name.  Call the `calculateSpeedAndPitch(fileName)` function with this file name.  The function produces:

* Classical mean speed calculated from all the discrete speeds sampled within the course
* USPA calculation ds/dt where dt<sub>competitionTime</sub> = t<sub>end</sub>-t<sub>start</sub>,
  and s<sub>courseLength</sub> = |s<sub>start</sub>-s<sub>end</sub>| for the first and last heights
  reported within the course window.
* Flight pitch in degrees at max speed
* Max speed
* Min speed

All speeds are reported in km/h.  Michael Cooper, USPA National Speed Skydiving Championship judged, explained
the "USPA calculation" as straight ds/dt in an email message from 04.Sep.2018.

### Explanation of the code in calculateSpeedAndPitchFrom(fileName)

1. Discard all source entries outside of the 2,700 to 1,700 m AGL course
1. Calculate the mean speed for the current jump (actual sum(N)/N and ds/dt for s0, s1
1. Resolve the max speed and pitch within the course
1. Return the results in a Series object, for later inclusion in a data frame with results from multiple jumps

### Specify the FlySight data file to analyze

In [33]:
DATA_SOURCE        = './data-sources/ciurana'
FLYSIGHT_DATA_FILE = '15-26-07.CSV'

In [42]:
def _discardDataOutsideCourse(fullFlightData, elevation = 0.00):
    height          = fullFlightData['hMSL', '(m)']
    descentVelocity = fullFlightData['velD', '(m/s)']
    courseStart     = fullFlightData['hMSL', '(m)'].max()
    
    return fullFlightData[(height <= courseStart) & (height >= COURSE_END+elevation) & (descentVelocity >= 0.0)]

In [43]:
def _calculateCourseSpeedUsing(flightData):
    """
    Returns course speed as ds/dt, and competitionTime in seconds
    """
    # TODO: figure out why pd.to_datetime() barfs
    startTime = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[0])
    endTime   = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[-1])

    startCourse = flightData['hMSL', '(m)'].values[0]
    endCourse   = flightData['hMSL', '(m)'].values[-1]

    competitionTime = (endTime-startTime).total_seconds()
    courseLength    = abs(endCourse-startCourse)

    return courseLength/competitionTime, competitionTime

In [44]:
def maxHorizontalSpeedFrom(flightData, maxVerticalSpeed):
    velN = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velN', '(m/s)']
    velE = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velE', '(m/s)']
    
    return math.sqrt(velN**2+velE**2)   # R vector

In [45]:
"""
    Adjusts the flight data to compensate for DZ elevation AMSL.
    
    flightData - the raw FlySight data frame
    elevation  - the elevation, in meters, to adjust
    
    All hMSL values in flightData will be offset by +elevation meters.
"""
def adjustElevation(flightData, elevation):
    flightData['hMSL', '(m)'] += elevation
    
    return flightData

In [47]:
def calculateSpeedAndPitchFor(fileName, elevation = 0.00):
    """
    Accepts a file name to a FlySight data file.
    
    Returns a Series with the results of a speed skydiving jump.
    """
    flightData  = adjustElevation(pd.read_csv(fileName, header = [0, 1]), elevation)
    flightData  = _discardDataOutsideCourse(flightData, elevation)
    sampledVelD = flightData['velD', '(m/s)'].mean()
    courseVelD, \
    courseTime  = _calculateCourseSpeedUsing(flightData)
    maxSpeed    = flightData['velD', '(m/s)'].max()
    minSpeed    = flightData['velD', '(m/s)'].min()
    
    pitchR       = math.atan(maxSpeed/maxHorizontalSpeedFrom(flightData, maxSpeed))
    maxSpeedTime = flightData[flightData['velD', '(m/s)'] == maxSpeed]['time', 'Unnamed: 0_level_1'].values[0]
    
    skydiveResults = pd.Series(
                        [
                            fileName,
                            maxSpeedTime,
                            3.6*sampledVelD,  # km/h; 3,600 seconds, 1,000 meters
                            3.6*courseVelD,
                            pitchR/DEG_IN_RAD,
                            3.6*maxSpeed,
                            3.6*minSpeed,
                            courseTime,
                        ],
                        [
                            'fileName',
                            'maxSpeedTime',
                            'sampledSpeed',
                            'courseSpeed',
                            'pitch',
                            'maxSpeed',
                            'minSpeed',
                            'courseTime',
                        ])

    return skydiveResults


calculateSpeedAndPitchFor(os.path.join(DATA_SOURCE, FLYSIGHT_DATA_FILE), elevation = DZ_AMSL)

fileName        ./data-sources/ciurana/15-26-07.CSV
maxSpeedTime                2019-03-29T23:40:12.40Z
sampledSpeed                                194.972
courseSpeed                                 143.961
pitch                                       67.3871
maxSpeed                                    359.892
minSpeed                                          0
courseTime                                       59
dtype: object

## Listing FlySight data files with valid data

This set up generates a list of FlySight data files ready for analysis.  It discards any warm up FlySight data files, those that show no significant changes in elevation MSL across the complete data set.

The list generator uses the `DATA_SOURCE` global variable.  [Change the value of `DATA_SOURCE`](#l_data-def) if necessary.

1. Generate the list of available data files
1. Discard FlySight warm up files by rejecting files without jump data (test:  minimal altitude changes)

![Scoring FlySight files list generation diagram](images/SSScoring-list-scoring-files.png)

In [ ]:
def listDataFilesIn(bucketPath):
    """
    Generate a sorted list of files available in a given bucketPath.
    Files names appear in reverse lexicographical order.
    """

    filesList = pd.Series([os.path.join(bucketPath, fileName) for fileName in sorted(os.listdir(bucketPath), reverse = True) if '.CSV' in fileName or '.csv' in fileName])
    
    return filesList;

In [ ]:
def _hasValidJumpData(fileName):
    flightData = pd.read_csv(fileName, header = [0, 1])['hMSL', '(m)']
    
    return flightData.std() >= VALID_MSL

In [ ]:
def selectValidFlySightFilesFrom(dataFiles):
    included = dataFiles.apply(_hasValidJumpData)
    
    return pd.Series(dataFiles)[included]

In [ ]:
dataFiles = selectValidFlySightFilesFrom(listDataFilesIn(DATA_SOURCE))
dataFiles

## Top speed and pitch analysis on all tracks in the data lake

Takes all the FlySight files in a bucket, detects the ones with valid data, and runs performance analysis over them.  Packs all the results in a data frame, then calculates:

* Average speed
* Max average speed

## Populate data sources

In [ ]:
DATA_LAKE   = './data-lake'
DATA_SOURCE = './data-sources/ciurana'

ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

### Analyze all files in the bucket

In [ ]:
allCompetitionJumps = selectValidFlySightFilesFrom(listDataFilesIn(DATA_SOURCE)).apply(calculateSpeedAndPitchFor)
allCompetitionJumps

### Summary of results

In [ ]:
summary = pd.Series(
            [
                len(allCompetitionJumps),
                allCompetitionJumps['maxSpeed'].mean(),
                allCompetitionJumps['maxSpeed'].max(),
                allCompetitionJumps['pitch'].max(),
            ],
            [
                'totalJumps',
                'meanSpeed',
                'maxSpeed',
                'maxPitch',
            ])

summary